# PoC #7: Meeting Notes → Action Items Converter 📝

## Business Use Case
Automatically extract action items, decisions, and next steps from meeting transcripts.

## Production Applications
- Meeting productivity tools
- Project management integration
- Executive assistant automation
- Team collaboration platforms

---

In [ ]:
!pip install -qU "ibm-watsonx-ai>=1.1.22"
print("✅ Ready")

In [ ]:
import getpass
from datetime import datetime
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

model = Model(
    model_id="ibm/granite-3-8b-instruct",
    credentials=Credentials(url="https://us-south.ml.cloud.ibm.com", api_key=getpass.getpass("API Key: ")),
    project_id=getpass.getpass("Project ID: "),
    params={GenParams.MAX_NEW_TOKENS: 400}
)
print("✅ Meeting analyzer ready")

In [ ]:
def extract_action_items(meeting_notes: str) -> dict:
    prompt = f"""Extract action items from these meeting notes. Format as numbered list with assignee and deadline if mentioned.

Meeting Notes:
{meeting_notes}

Action Items:"""
    
    actions = model.generate_text(prompt=prompt)
    
    return {
        "meeting_date": datetime.now().strftime("%Y-%m-%d"),
        "original_notes": meeting_notes,
        "action_items": actions.strip(),
        "num_actions": len([line for line in actions.split('\n') if line.strip().startswith(('1', '2', '3', '4', '5'))])
    }

In [ ]:
# Sample meeting notes
meeting_transcript = """Product Strategy Meeting - 2024-11-18

Attendees: Sarah (PM), Mike (Engineering), Lisa (Design), Tom (Marketing)

Discussion:
- Mike mentioned the API integration is behind schedule. He'll need 2 more weeks.
- Lisa presented new UI mockups. Everyone approved. She'll finalize by Friday.
- Tom proposed launching beta program next month. Sarah agreed and will draft requirements.
- Decision: Move launch date from Dec 1 to Dec 15 to accommodate API delays.
- Mike to coordinate with DevOps on deployment pipeline.
- Sarah to schedule follow-up meeting for Nov 25.
"""

result = extract_action_items(meeting_transcript)

print(f"\n{'='*80}\n📅 MEETING SUMMARY - {result['meeting_date']}\n{'='*80}")
print(f"\n🎯 ACTION ITEMS ({result['num_actions']} total):\n")
print(result['action_items'])
print(f"\n{'='*80}")

In [ ]:
# Batch processing
meetings = {
    "Sprint Planning": "Discussed Q1 roadmap. John to create JIRA tickets by EOD. Maria to review designs by Wed.",
    "Client Call": "Client requested new feature. Tom to send proposal by Friday. Sarah to follow up next week."
}

for meeting_name, notes in meetings.items():
    result = extract_action_items(notes)
    print(f"\n📌 {meeting_name}: {result['num_actions']} action(s)")

In [ ]:
# Your meeting
print("Enter your meeting notes (press Enter twice when done):\n")
lines = []
while True:
    line = input()
    if line == "": break
    lines.append(line)

your_notes = "\n".join(lines)
if your_notes.strip():
    result = extract_action_items(your_notes)
    print(f"\n{'='*80}\n🎯 EXTRACTED ACTION ITEMS\n{'='*80}\n")
    print(result['action_items'])

---

## Production Benefits
- ⏱️ Save 30 min per meeting
- ✅ Never miss action items
- 📊 Track accountability
- 🔗 Auto-create Jira/Asana tasks

### Integration Ideas
1. Zoom/Teams transcript API
2. Slack notification bot
3. Calendar integration
4. Project management tools

---